Easy example of using CNN in PyTorch

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = '../input'
train_dir = data_dir + '/train/train/'
test_dir = data_dir + '/test/test/'

In [3]:
labels = pd.read_csv("../input/train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [4]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        
        img_path = os.path.join(self.data_dir, img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image, label

In [5]:
epochs = 25
batch_size = 20
device = torch.device('cuda:0')

In [6]:
data_transf = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
train_data = ImageData(df = labels, data_dir = train_dir, transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size = batch_size)

In [7]:
class Network(nn.Module): 
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(720, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [8]:
net = Network().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [9]:
%%time
# Train model
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward
        outputs = net(images)
        loss = loss_func(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 500 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))

Epoch [1/25], Loss: 0.0986
Epoch [2/25], Loss: 0.0877
Epoch [3/25], Loss: 0.0736
Epoch [4/25], Loss: 0.0612
Epoch [5/25], Loss: 0.0649
Epoch [6/25], Loss: 0.0985
Epoch [7/25], Loss: 0.0307
Epoch [8/25], Loss: 0.0518
Epoch [9/25], Loss: 0.1849
Epoch [10/25], Loss: 0.0741
Epoch [11/25], Loss: 0.0268
Epoch [12/25], Loss: 0.0338
Epoch [13/25], Loss: 0.1144
Epoch [14/25], Loss: 0.0101
Epoch [15/25], Loss: 0.1189
Epoch [16/25], Loss: 0.0099
Epoch [17/25], Loss: 0.0447
Epoch [18/25], Loss: 0.0178
Epoch [19/25], Loss: 0.0114
Epoch [20/25], Loss: 0.0751
Epoch [21/25], Loss: 0.0197
Epoch [22/25], Loss: 0.0194
Epoch [23/25], Loss: 0.0071
Epoch [24/25], Loss: 0.0041
Epoch [25/25], Loss: 0.1495
CPU times: user 4min 32s, sys: 29.2 s, total: 5min 1s
Wall time: 6min 6s


In [10]:
submit = pd.read_csv('../input/sample_submission.csv')
test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [11]:
predict = []
for batch_i, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    output = net(data)
    
    _, pred = torch.max(output.data, 1)
    predict.append(int(pred))
    
submit['has_cactus'] = predict
submit.to_csv('submission.csv', index=False)

In [12]:
submit.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,1
1,0017242f54ececa4512b4d7937d1e21e.jpg,1
2,001ee6d8564003107853118ab87df407.jpg,0
3,002e175c3c1e060769475f52182583d0.jpg,0
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0
